In [1]:
pip install --upgrade ibm-watson

     |████████████████████████████████| 389kB 2.8MB/s 
     |████████████████████████████████| 204kB 7.7MB/s 
  Created wheel for ibm-watson: filename=ibm_watson-4.7.1-cp36-none-any.whl size=379133 sha256=5106f813ea58e740cdbd558742ac7b04caf6cc0bc86b41dad2b4ae6419e16ae8
  Stored in directory: /root/.cache/pip/wheels/6e/14/69/dbbd573a3bab3bf64984572284f13f174f430038308abdd73c
  Created wheel for ibm-cloud-sdk-core: filename=ibm_cloud_sdk_core-1.7.3-cp36-none-any.whl size=45945 sha256=926f59f1158b2f7c658ad1c4ae01405dfa5f858a2fed8d825cf91fbb4981e903
  Stored in directory: /root/.cache/pip/wheels/34/6e/58/589e0f841c2fae9dad99630d78ddc7a60c5c7663a16a39cdbb
Successfully built ibm-watson ibm-cloud-sdk-core


In [2]:
pip install textblob vaderSentiment watson-developer-cloud

     |████████████████████████████████| 133kB 2.7MB/s 
     |████████████████████████████████| 256kB 8.6MB/s 
  Created wheel for watson-developer-cloud: filename=watson_developer_cloud-2.10.1-cp36-none-any.whl size=252814 sha256=828bb2affd1b590ddecc50736b90d3799f4a6d1f2141ce2a642d71e6ab337ced
  Stored in directory: /root/.cache/pip/wheels/ea/a7/19/fac0a408a586265eb374005308a7553d4494ab41b2fd88f5bc
Successfully built watson-developer-cloud


In [3]:
from ibm_watson import DiscoveryV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator 
authenticators = IAMAuthenticator("jpRrhH72mpjpTTyEW4HcrnjRnaLcUPP7dvGk2icU8cZN",url="https://iam.cloud.ibm.com/identity/token")
discovery = DiscoveryV1(version="2018-08-01",authenticator=authenticators)
discovery.set_service_url("https://api.eu-gb.natural-language-understanding.watson.cloud.ibm.com/instances/f26d61bf-45d3-4f9b-a219-a0f760d0ba71")

In [4]:
import numpy as np 
import pandas as pd
from textblob import TextBlob 
#from watson_developer_cloud import DiscoveryV1
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from ibm_watson.natural_language_understanding_v1 import Features,EntitiesOptions,KeywordsOptions,SentimentOptions,CategoriesOptions
from ibm_watson import NaturalLanguageUnderstandingV1

In [5]:
total_csv = pd.read_csv("/content/drive/My Drive/Data science resources/nlp/total.csv")
len(total_csv)

6486

In [6]:
total_csv.columns

Index(['user', 'time', 'text', 'favourite_count', 'retweet_count', 'lang',
       'description', 'country', 'tweet_type', 'user_type'],
      dtype='object')

In [7]:
total_csv = total_csv.drop_duplicates(subset=['text'])

In [8]:
len(total_csv)

5532

In [9]:
import re 
def remove_unwanted(x):
  global re
  x = re.sub(r'@[A-Za-z0-9]+','',x)
  x = re.sub(r'#','',x)
  x = re.sub(r'https?\/\/\s+','',x)
  return x

In [10]:
total_csv['text'] = total_csv['text'].apply(remove_unwanted)

In [11]:
total_csv.drop_duplicates(subset=['text'],inplace=True)

In [12]:
len(total_csv)

5531

In [13]:
# total_csv["text"]==

In [14]:
# def remove_retweets(x):
#   if x[:2]== 'rt':
#     return 1 
#   else:
#     return 0

In [16]:
#total_csv['retweets_text'] = total_csv['text'].apply(remove_retweets)

In [19]:
#total_csv = total_csv.loc[total_csv['tweet_type']!="RT",:]

In [20]:
analyser = SentimentIntensityAnalyzer()
natural_language_understanding = NaturalLanguageUnderstandingV1(version="2018-11-16",authenticator=authenticators)
natural_language_understanding.set_service_url("https://api.eu-gb.natural-language-understanding.watson.cloud.ibm.com/instances/f26d61bf-45d3-4f9b-a219-a0f760d0ba71")
#DiscoveryV1(version="2018-08-01",authenticator=authenticators)

In [21]:
def analyze_text(input_text,analyzer):
  if analyzer =='VADER':
    result = analyser.polarity_scores(input_text)
    score = result['compound']
  elif analyzer =='textblob':
    score = TextBlob(input_text).sentiment.polarity
  else:
    response = natural_language_understanding.analyze(text=input_text,features=Features(sentiment=SentimentOptions()),language='en').get_result()
    score = response.get('sentiment').get('document').get('score')
  return score 



In [23]:
total_csv['vader_score'] =total_csv['text'].apply(analyze_text,analyzer='VADER')
total_csv['textblob'] = total_csv['text'].apply(analyze_text,analyzer='textblob')
# total_csv['ibm_score'] = total_csv['text'].apply(analyze_text,analyzer='ibm')